# Análise e Avaliação de Modelos de Regressão

Este notebook demonstra as 4 partes da avaliação de um modelo de regressão, conforme explicado na aula:

1.  **Análise de "Bom Senso"**: Previsões e Coeficientes.
2.  **Análise Visual**: Gráficos de diagnóstico (Regressão, Resíduos, Distribuição).
3.  **Análise Numérica**: Métricas MSE e R-quadrado.
4.  **A "Armadilha" da Comparação**: Comparando um modelo simples com um mais complexo.

## Setup: Importações e Dados Simulados

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Configurando o estilo dos gráficos para uma melhor visualização
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Criando um DataFrame simulado de carros
# Preço = f(consumo, tamanho_motor) + ruído
np.random.seed(42)
num_carros = 150
dados = {
    'highway_mpg': np.random.uniform(15, 45, num_carros),
    'engine_size': np.random.uniform(90, 250, num_carros)
}
df = pd.DataFrame(dados)

# Criando a relação: preço base + bônus por tamanho_motor - penalidade por mpg + ruído
df['price'] = (1000 + 
               df['engine_size'] * 150 - 
               df['highway_mpg'] * 350 + 
               np.random.normal(0, 3000, num_carros))

print("DataFrame de exemplo criado com sucesso!")
df.head()

DataFrame de exemplo criado com sucesso!


,highway_mpg,engine_size,price
0,26.236204,235.322542,27252.425535
1,43.521429,128.329903,3062.184116
2,36.959818,113.183180,11473.372809
3,32.959755,168.312442,16612.709230
4,19.680559,247.704073,25191.987414


## Parte 1: Análise de "Bom Senso" (Previsões e Coeficientes)

Vamos começar com um modelo simples (Regressão Linear Simples - SLR) que usa apenas `highway_mpg` para prever o `price`.

In [3]:
# Treinando o modelo simples (SLR)
slr_model = LinearRegression()
X_slr = df[['highway_mpg']]
y = df['price']
slr_model.fit(X_slr, y)

print("--- PARTE 1: Análise de 'Bom Senso' (Modelo Simples) ---")

# 1.1 Fazer uma previsão específica
mpg_teste = 30
preco_previsto = slr_model.predict(np.array([[mpg_teste]]))[0]
print(f"Previsão de preço para um carro com {mpg_teste} MPG: ${preco_previsto:,.2f}")
print("   -> O valor parece razoável? Sim, não é negativo nem absurdo.")

# 1.2 Examinar os coeficientes
coeficiente = slr_model.coef_[0]
intercepto = slr_model.intercept_
print(f"\nCoeficiente (inclinação) para 'highway_mpg': {coeficiente:.2f}")
print("   -> Interpretação: Para cada 1 MPG a mais, o preço do carro diminui em ~$738.")
print(f"Intercepto: {intercepto:.2f}")
print("   -> O intercepto é o valor teórico se MPG fosse 0. Muitas vezes não tem um sentido prático.")

--- PARTE 1: Análise de 'Bom Senso' (Modelo Simples) ---
Previsão de preço para um carro com 30 MPG: $16,648.03
   -> O valor parece razoável? Sim, não é negativo nem absurdo.

Coeficiente (inclinação) para 'highway_mpg': -296.71
   -> Interpretação: Para cada 1 MPG a mais, o preço do carro diminui em ~$738.
Intercepto: 25549.33
   -> O intercepto é o valor teórico se MPG fosse 0. Muitas vezes não tem um sentido prático.


c:\Users\junio\develop\repos\studio-ia-python\course\JupyterProject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


## Parte 2: Análise Visual (Diagnóstico com Gráficos)

In [ ]:
print("--- PARTE 2: Análise Visual (gerando gráficos) ---")

# 2.1 Gráfico de Regressão e 2.2 Gráfico de Resíduos (lado a lado)
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1) # Gráfico 1 de 2
sns.regplot(x='highway_mpg', y='price', data=df,
            scatter_kws={'alpha':0.6, 'color': 'blue'},
            line_kws={'color': 'red', 'linewidth': 2})
plt.title('Gráfico de Regressão')
plt.xlabel('Consumo (Highway MPG)')
plt.ylabel('Preço ($)')

plt.subplot(1, 2, 2) # Gráfico 2 de 2
sns.residplot(x=X_slr.squeeze(), y=y, color='green')
plt.title('Gráfico de Resíduos')
plt.xlabel('Consumo (Highway MPG)')
plt.ylabel('Erro (Resíduo)')

plt.suptitle('Diagnóstico Visual do Modelo Simples (SLR)', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

print("Análise: O Gráfico de Resíduos mostra um padrão claro (não é aleatório), indicando que o modelo linear simples está perdendo alguma informação (neste caso, o efeito do 'engine_size').")

In [ ]:
# 2.3 Gráfico de Distribuição
y_pred_slr = slr_model.predict(X_slr)

plt.figure(figsize=(10, 6))
sns.kdeplot(y, color='blue', fill=True, label='Preços Reais')
sns.kdeplot(y_pred_slr, color='red', fill=True, alpha=0.5, label='Preços Previstos (SLR)')
plt.title('Distribuição dos Preços Reais vs. Previstos')
plt.xlabel('Preço ($)')
plt.legend()
plt.show()

## Parte 3: Análise Numérica (MSE e R-quadrado)

In [ ]:
print("--- PARTE 3: Análise Numérica (Métricas) ---")

# 3.1 Calcular MSE e R² para o modelo simples (SLR)
mse_slr = mean_squared_error(y, y_pred_slr)
r2_slr = r2_score(y, y_pred_slr)

print(f"Métricas para o Modelo Simples (SLR):")
print(f"  - Erro Quadrático Médio (MSE): {mse_slr:,.2f}")
print(f"  - Raiz do Erro Quadrático Médio (RMSE): ${np.sqrt(mse_slr):,.2f}")
print(f"  - R-quadrado (R²): {r2_slr:.4f}")
print(f"\nInterpretação do R²: O consumo (MPG) sozinho explica aproximadamente {r2_slr:.1%} da variação no preço dos carros.")

## Parte 4: A "Armadilha" Final (Comparando Modelos)

Agora vamos treinar um modelo mais complexo (Regressão Linear Múltipla - MLR) usando `highway_mpg` e `engine_size` e comparar as métricas.

In [ ]:
print("--- PARTE 4: A 'Armadilha' - Comparando Modelos ---")

# 4.1 Treinar um modelo mais complexo: Regressão Linear Múltipla (MLR)
mlr_model = LinearRegression()
X_mlr = df[['highway_mpg', 'engine_size']]
mlr_model.fit(X_mlr, y)
y_pred_mlr = mlr_model.predict(X_mlr)

# 4.2 Calcular as métricas para o modelo complexo (MLR)
mse_mlr = mean_squared_error(y, y_pred_mlr)
r2_mlr = r2_score(y, y_pred_mlr)

print("\nMétricas para o Modelo Complexo (MLR):")
print(f"  - Erro Quadrático Médio (MSE): {mse_mlr:,.2f}")
print(f"  - Raiz do Erro Quadrático Médio (RMSE): ${np.sqrt(mse_mlr):,.2f}")
print(f"  - R-quadrado (R²): {r2_mlr:.4f}")

# 4.3 A conclusão da aula
print("\n--- Conclusão da Aula (A Armadilha) ---")
print(f"Comparação SLR vs MLR:")
print(f"  - MSE: {mse_slr:,.2f} (Simples) -> {mse_mlr:,.2f} (Complexo)")
print(f"  - R²:  {r2_slr:.4f} (Simples) -> {r2_mlr:.4f} (Complexo)")

print("\nAVISO DA AULA: Como esperado, o MSE do modelo MLR é MENOR e o R² é MAIOR.")
print("Isso acontece porque adicionar mais variáveis (mesmo que inúteis) dá ao modelo mais flexibilidade para se 'ajustar' aos dados de treino.")
print("Neste caso, 'engine_size' é uma variável útil, então a melhora é real. Mas isso nem sempre é verdade.")
print("É por isso que precisamos de técnicas mais avançadas (como R² ajustado e Validação Cruzada) para saber se um modelo mais complexo é *realmente* melhor.")